In [361]:
import torch
import importlib
import matplotlib.pyplot as plt
import src.physics as physics
importlib.reload(physics)

<module 'src.physics' from '/Users/aidanbx/CS/EINCASM/src/physics.py'>

# Activation and Flow

Each cell has a set of channel equal in length to the kernel (minus 1 for the origin cell) that denote the weight of connecting edges between that cell and the cell offset from the origin by the amounts specified in the kernel.

The kernel is to be symmetric and start with the origin [0,0]. This means that by rolling the non-origin kernel by half of its length (length(kernel)-1) leaves the offsets in the order of their inverses. This mirrors a kernel. 

Each cell generates a single activation (though it is easy to allow for len(kernel) activations). This activation is positive or negative, as are the weights. If the result of multiplying the activation by the weights is negative, this indicates inverted flow (taking from a neighbor). To compute this, we can simply roll all negative flows such that they are in inverted positions and subtract (so add) them from the flows of their neighbors in the inverted direction (taking from your neighbor is the same as them giving something to you). 

At this point, the flow tensor should only include positive flows and previously negative flows should be set to 0. With this, we can evenly distribute the capital of each cell across its flows and update the capital of each cells' neighbors. This doesn't necessitate that all capital be moved from the cell, the activation could be small and flows associated with the origin (position 0) could be high, meaning capital is given to yourself in the next timestep.  

In [307]:
# Define the kernel and world tensors
# Each kernel element is a shift along the 0 and 1 axes of the world for each channel/layer
kernel = torch.tensor([
    [0, 0],     # ORIGIN
    [-1, 0],     # UP
    [0, 1.0],   # RIGHT
    [1, 0],     # DOWN
    [0, -1]    # LEFT
])

muscle_radii = torch.tensor([
    [[-1,  1, 0],      # (0,0) ORIGIN
     [-0.8,  0.8, 0],  
     [0, 0, 0]],  

    [[-0.6, 0.6, 0],    # UP
     [-0.4, 0.4, 0],   
     [0, 0, 0]],   

    [[-0.2, 0.2, 0],    # RIGHT
     [0, 0, 0],
     [0, 0, 0]],
    
    [[0.2, -0.2, 0],    # DOWN
     [0.4, -0.4, 0],
     [0, 0, 0]],
    
    [[0.6, -0.6, 0],    # LEFT
     [0.8, -0.8, 0],
     [0, 0, 0]]
])

activations = torch.tensor([
    [1, 1, 0],
    [1, 1, 0],
    [0, 0, 0]
])

capital = torch.tensor([
    [4, 3.0, 0],
    [2, 1, 0],
    [0, 0, 0]
])
before = capital.clone()

flow_efficiency = torch.tensor([
    [0.8, 0.8, 0.8],
    [0.8, 0.8, 0.8],
    [0.8, 0.8, 0.8]
])

physics.activate_muscles_and_flow(capital, muscle_radii, activations, flow_efficiency, kernel)

tensor([[3.0000, 2.1600, 0.6400],
        [1.2320, 0.3200, 0.8320],
        [0.2560, 0.3200, 0.0000]])

# Muscle Growth

Cell growth in EINCASM requires an exchange of a cell's capital to slowly adjust the magnitude of a muscle oriented in a specific direction. If the change in magnitude of a muscle is positive, capital is taken from the cell with an adjustment for "heat loss" reflecting the efficiency of the process. If the change in magnitude is negative, capital, minus a loss, is returned to the cell. 

Muscles, as explained in #Activation and Flow, have an orientation and can be positive or negative. They are encoded in the world tensor as a set of channels where each is associated via order of index with the kernel, which contains offsets. If EINCASM is set up to only have a singular positive or negative activation per cell, then negative and positive muscles always have inverted flows in respect to each other. Because a cell can only change its muscle weights slowly and at a cost of wasted capital, it is to their advantage to choose an orientation and stick to it. 

There a multiple ways muscle growth can be accounted for in EINCASM. The default set up is to have the muscle channels denote the radius and charge of a muscle fiber. The output of the physiology of each cell indicates a change in radius, but the cost of growth is proportional to the cross-sectional area of the new muscle radius minus the original. This set up is chosen to enable flexibilty at a wider range of muscle magnitude scales without drastic changes to weights within the physiology (a neural network). The same cell that controls the small, exploratory muscle fibers/transport tubes can control the very large, established muscles next to large nutrient sources. 

### Obstacles and Obstacle Masks

Obstacles are currently handled during muscle growth rather than flow. No muscle can grow in the direction of an adjacent obstacle. It is possible to implement this during flow where capital rebounds to the origin cell, but handling this during muscle growth is simpler.

In [358]:
kernel = torch.tensor([
    [0, 0],     # ORIGIN
    [-1, 0],    # UP
    [0, 1.0],   # RIGHT
    [1, 0],     # DOWN
    [0, -1]     # LEFT
])

muscle_radii = torch.tensor([
    [[0,  0],   # ORIGIN
     [0,  0]],  

   [[-1, -4.0], # UP
    [1, 4]],   

   [[2, 1],     # RIGHT
    [2, 5]],
    
   [[0.0, 0.0], # DOWN
    [0.0, 0.0]],
    
   [[0.0, 0.0], # LEFT
    [0.0, 0.0]]
])

radii_deltas = torch.tensor([
    [[0, 0],        # ORIGIN
     [0, 0]],

    [[1.0, 2.0],    # UP
     [-3.0, 1.0]],

    [[1.0, -1.0],   # RIGHT
     [-4.0, 2.0]],

    [[0.0, 0.0],    # DOWN
     [0.0, 0.0]],

    [[0.0, 0.0],    # LEFT
     [0.0, 0.0]]
])     

capital = torch.tensor([
    [5.0, 3.0],
    [4.0, 0.0]
])

growth_efficiency = torch.tensor([
    [0.8, 0.85],
    [0.9, 1.0]
])

open_cells = torch.tensor([
    [1, 1],
    [1, 0],
], dtype=torch.bool) # 1 = free, 0 = obstacle

directional_masks = torch.ones_like(muscle_radii, dtype=torch.bool)

for i in range(kernel.shape[0]):
    directional_masks[i] = torch.roll(open_cells, shifts=tuple(map(int, -kernel[i])), dims=[0, 1])

muscle_masks = directional_masks&open_cells

In [362]:
physics.grow_muscle_csa(muscle_radii, radii_deltas, capital, growth_efficiency, muscle_masks, open_cells)

(tensor([[[ 0.0000,  0.0000],
          [ 0.0000,  0.0000]],
 
         [[ 0.2891,  0.0000],
          [-2.1448,  0.0000]],
 
         [[ 2.9251, -0.9220],
          [ 0.0000,  0.0000]],
 
         [[ 0.0000,  0.0000],
          [ 0.0000,  0.0000]],
 
         [[ 0.0000,  0.0000],
          [ 0.0000,  0.0000]]]),
 tensor([[0.0000, 2.8500],
         [0.0000, 0.0000]]))